In [ ]:
# Prep
import pandas as pd
import re
import os
from os.path import join

data_dir = ''
parl_dir = join(data_dir, 'dk_parl')
eurob_path = join(data_dir, 'eurobarometer-96_dk_subset.csv')
parl_path = join(parl_dir, 'dkparl_125_20220825.txt')

with open(parl_path, 'r', encoding = 'utf-8') as f:
    parl_ref = f.read()
    
eurob = pd.read_csv(eurob_path)

## Brug af egne funktioner

Fordelen ved at skrive egne funktioner er, at man kan lave funktioner, der er helt specifikke til de datasæt, man sidder med, og de datahåndtering, som man har brug for.

Egne funktioner kan have flere anvendelser, fx:
- Specifik datahåndteringsfunktion, som skal bruges på flere kolonner/datasæt (fx med mange betingelser)
- Behandling af bestemt type fil (fx tekstfil)
- Konvertering fra en datastruktur til en anden

### Funktion til at behandle tekst

Tidligere arbejdede vi lidt med et referat fra et møde i Folketinget. Strukturen i dokumentet tillader, at vi kan dele det op i dagsordenspunkter:

In [ ]:
import re

point_pattern = re.compile("---+")

parlref_points = point_pattern.split(parl_ref)

parlref_points[2]

Derfra kunne vi være interesseret i at udlede titlerne for punkterne. For at gøre dette laver laver vi en dictionary for hvert punkt (de to første elementer i listen er ikke dagsordenspunkter).

In [ ]:
point_t_re = re.compile("(?<=\n)\d\) .*?(?=:?\n)")

parlref_pointsdata = []

for point in parlref_points[2:len(parlref_points)]:
    if point_t_re.search(point):
        point_t = point_t_re.search(point).group(0)
    else:
        point_t = ""
    
    point_dict = {"title": point_t, "text": point}
    
    parlref_pointsdata.append(point_dict)

In [ ]:
parlref_pointsdata[0]

In [ ]:
[point.get('title') for point in parlref_pointsdata]

Ovenstående kunne også laves til en funktion. Med denne har man en funktion, der kan anvendes på referater af møder i Folketinget generelt:

In [ ]:
def create_parlpointsdata(parlreftext):
    
    #regex
    point_pattern = re.compile("---+")
    point_t_re = re.compile("(?<=\n)\d\) .*?(?=:?\n)")
    
    #split into points
    parlref_points = point_pattern.split(parlreftext)
    
    #create dataset (key for title and text)
    parlref_pointsdata = []

    for point in parlref_points[2:len(parlref_points)]:
        if point_t_re.search(point):
            point_t = point_t_re.search(point).group(0)
        else:
            point_t = ""

        point_dict = {"title": point_t, "text": point}

        parlref_pointsdata.append(point_dict)
        
    return(parlref_pointsdata)

In [ ]:
parlref_points_test = create_parlpointsdata(parl_ref)

parlref_points_test[0]

### Brug af egne funktioner i pandas

Pandas tillader også, at man kan anvende sine egne funktioner. Dette gøres med metoden `apply()`.

`.apply()` tager en funktion som input. Denne funktion skal gerne virke med en enkelt værdi som input (en værdi fra en kolonne), da `.apply()` så sørger for at anvende den på alle værdierne i kolonnen.

In [ ]:
eurob.head()

In [ ]:
def replacer(cat):
    if cat == "Not mentioned":
        value = 0
    else:
        value = 1
        
    return(value)
        
eurob['qb3_1_bin'] = eurob['qb3_1'].apply(replacer)

eurob.head()